# Homework VI
**Hello!** Welcome to the Reinforcement Learning assignment. To start, let's import some packages. You'll need to import the first environment we'll work on today: a sepsis treatment problem.

In [ ]:
import numpy as np # you should know what this does
import scipy # you should also know this
import gymnasium as gym # gymnasium is the maintained version of what used to be OpenAI gym, a very popular package
# whose format and included environments have been very influential on the practice of Reinforcement Learning.
import torch # you probably know this
from torch import nn # you probably know this
import torch.nn.functional as F # you probably know this 
import time # you probably know this
import matplotlib.pyplot as plt

from collections import namedtuple, deque # used for transition buffer
from itertools import count # used for transition buffer
import random # used for transition buffer

# For the sepsis environment
import yaml # the sepsis problem needs this. don't ask me what it is, i don't know. pip install pyyaml
import sepsis_mdp_master.mdps as sepsis_mdp # the process given to us by a master's student here

: 

### JUST RUN THIS, IT'LL MAKE STUFF WORK

In [ ]:
config = {                                      # used in the definition of certain functions
"time_limit" : 1000,                            # maximum temporal length of a process - 30 [days] is used for sepsis
"sepsis_time_limit" : 30,                       # time limit for sepsis environment
"epsilon" : 0.15,                               # used for epsilon-greedy algorithms
"gamma" : 0.999,                                # reward discount rate
"alpha" : 0.15,                                 # used for averaging
"survival_rate_sample_size" : 1000,             # number of samples 
"stochastic" : True,                            # this is changed during the evaluation of sepsis policies
"LR" : 0.001,                                   # Learning rate for Q-Networks. Used for Problems >= 4
"tabular_q_mode" : "Monte_Carlo",               # alternative: "Expected_SARSA". You'll be writing the Monte_Carlo version.
"q_network_stochasticity" : "epsilon_greedy",   # policy method used for q network
"q_network_batch_size" : 128,                   # number of transitions used per training batch for q network
"q_network_hidden_layer_width" : 128,           # width of q network hidden layers
"transition_buffer_size" : 10000,               # size of buffer of transitions used for q network
"q_network_epsilon_decay" : 0.999,              # rate of decay of q_network.epsilon for q_network training
"batched_td_method" : "SARSA",                  # q network training method (alternative: "argmax")
"train_sepsis" : False,                         # contols part of run_episode
"visualize_learning" : True,                    # sets whether to visualize ALL episodes during training
"visualization_frequency" : 25,                 # sets how often to visualize episodes during training
}

# Sepsis specific:
def get_size(space): # I couldn't figure out a better way to do this, you just can't query this quantity easily
    # from certain gym environments (it has to do with gym.spaces.Multidiscrete, if you must know.)
    if type(space) == gym.spaces.Discrete:
        return gym.spaces.utils.flatten_space(space).shape[0]
    length = 1
    for value in space:
        length = length * value.n
    return length

def binary_action_to_int(action): # converts a 3-length binary [stored as integer] list into an integer - sepsis
    return action[0] * 4 + action[1] * 2 + action[2] * 1

def action_index_to_binary(index): # converts an integer into a 3-length binary list - sepsis
    action = np.zeros(3)
    action[0] = index // 4
    index = index % 4
    action[1] = index // 2
    index = index % 2
    action[2] = index // 1
    action = action.astype(int)
    return action

class random_policy_sepsis: # the sepsis environment is quite nice but it really is particular!
    def __init__(self, env):
        self.env = env
        self.num_actions = get_size(env.action_space)

    def get_action(self, state, stochastic = True):
        selection = np.random.randint(self.num_actions)
        action = action_index_to_binary(selection)
        return action
    
# General:

class random_policy: # general purpose random policy for any (in-specification) gym environment
    def __init__(self, environment):
        self.env = environment

    def get_action(self, state=None, stochastic = True):
        return self.env.action_space.sample()


<!-- The first problem here will be to write the loop below. -->

## Problem 1 : 10 points
First, we need to set up our environment. For a new problem, this is usually pretty difficult, especially creating the reward function (as we discussed in class).

Fortunately, there are a lot of off-the-shelf environments for us to use, many in the `gymnasium` package imported above. So, let's try one.

You'll need to make a loop to iterate through your environment. Luckily for you we'll give you an outline on what you'll need to do! 

 1. To initialize an environment using `gymnasium` (henceforth `gym`), you'll run `env = gym.make('EnvironmentName-vN')`, where `N` is a version number for the environment. Add the argument `render_mode = "human"` to the initialization to watch your process in action (it will open in a new window so you'll need to open it).

 2. The specification for `gym` includes a few variables we won't use - `info` and `truncated`. We will need to assign these variables when they come from a `gym` function.

 3. To get the initial state of the process, run `state, info = env.reset()`.

 4. As we spoke about in class, after the first state (obtained from `env.reset()`), every state is randomly selected based upon the action of the agent. You'll use a function of the form `agent.get_action(state)` to get a state from your agent.
 
 5. To get the next state in the process, you'll call `env.step(action)`, which returns 5 values: `state, reward, terminated, truncated, info`.

 6. To get a policy that you can verify your loop with, call `mountain_car_random = random_policy(env)`.

 <!-- 7. Store the return fom the process in a variable called `episode_return`. -->

For this problem, you should use the environment `"MountainCar-v0"`.

In [ ]:
env = gym.make("MountainCar-v0", render_mode = "human") # knockout

mountain_car_random = random_policy(env) # knockout

# You'll use this configuration value to define when your episodes will end
problem_1_config = config.copy()
problem_1_config["time_limit"] = 50

# Run 5 episodes
for episode in range(5):
    # Initialize your process here!
    state, info = env.reset() # knockout
    problem_1_terminated = 0 
    timestep = 0
    reward_total = 0

    while problem_1_terminated != True and timestep < problem_1_config["time_limit"]: 

        action = mountain_car_random.get_action(state) # knockout
        state, reward, problem_1_terminated, truncated, info = env.step(action) # knockout

        timestep = timestep + 1
        reward_total = reward_total + reward
    print("The total reward in episode ", episode ," was ", reward_total )

env.close() # close your environments when you're done with them! 
# Don't worry about closing an environment you're going to reuse, though - env.reset() will reset everything for you.

The rewards won't be going up since we aren't learning yet, but as long as this runs and returns some rewards you're golden!

This is the basic structure that we're going to use for the rest of this assignment. Or, more accurately, the internal loop is. In the next problem, we'll build a generalized version.

## Problem 2 : 20 points

As your next task, you're going to need to define a function called `run_episode`, which takes in a `gym` environment, a `policy`, and a `configuration dictionary` which contains miscellaneous variables such as our `gamma` (discount factor).

run_episode should return a list called `trajectory`, where for each timestep there is an entry of the form 

`[state, action, reward, return_to_go]`. 

Here, `return_to_go` is the *discounted* return from the current timestep until the end of the episode (i.e. the sum of all future rewards discounted appropriately). Test it with `random_policy.get_action(state)`. Make to terminate the episode when the time limit (number of actions/steps), `config["time_limit]`, has been passed! (hint: the total return for an episode should be stored in `trajectory[0][3]`)

You should have a loop that moves through the environment and stores the `state, action, and reward` for each action. Then you should have a loop that retroactively finds the `return_to_go` for each step using the future rewards. 


### Problem 2a : 5/20 points
Write the formula for the return to go from a timestep $t$ to the end of an episode below. Recall that the **reward** $r$ represents the response of the environment to a particular action, while the **return** $R$ is an aggregation of these rewards.

$$\tau = ((s_{0}, a_{0}), (s_{1}, a_{1}), (s_{2}, a_{2}), (s_{3}, a_{3}) \ldots (s_{t}, a_{t}))$$

$$R(\tau, t) = $$

### Problem 2b : 15/20 points
Write the function `run_episode` below

In [ ]:
def run_episode(environment, policy, configuration):

    '''
    This function runs a single episode through the given environment using the given policy and configuration settings 
    ----------
    Environment : Gym Environment
        The environment you will be using to get states and rewards based off of your actions
    Policy : Policy object
        A given strategy for determing what actions to take while in a given state, this could be random,
        q-learning, or even a neural network
    Configuration: Dictionary
        A dictionary containing all of the needed settings for the environment and/or policy. For example our discounting factor, "gamma",
        is contained here and we access to be able to calculate our reward_to_go's
    Returns
    -------
    Trajectory: 2D List
        This is a list of lists, each row corresponds to a single timestep and contains the state, action taken, reward given, and reward_to_go
        Reward_to_go is the expected reward at that timestep considering the rewards that follow and discounting them appropriately.
    
    
    '''

    state, info = environment.reset()                                                                   
    terminated = 0                                                                                     
    truncated = 0                                                                                       
    gamma = configuration["gamma"]                                                                      
    trajectory = []                                                                                     
    timestep = 0                                                                                        
    # TODO Once you have all of your variables defined, create a loop that runs until the episode should end
    ########################
    

    ########################
        # This code handles some annoying edge cases you'll hit later - put this code in after you assign an action,
        if torch.is_tensor(action):
            action = action.flatten().cpu().detach().numpy()[0]
        if configuration["train_sepsis"] == True and type(action) == int:
            new_state, reward, terminated, truncated, info = environment.step(action_index_to_binary(action))
        elif configuration["train_sepsis"] == True and type(action) == np.int64:
            action = int(action)
            new_state, reward, terminated, truncated, info = environment.step(action_index_to_binary(action))

        # TODO write the rest of the code for the episode here, in the else statement:
        else:


        # TODO Update timestep, state and record the current timesteps trajectory
        #######################################    


        ####################################### 

    # TODO Once your episode is over, calculate the return to go for each timestep.
    #######################################  



    #######################################  

    return trajectory

Let's test your `run_episode` function in another classic environment: CartPole!
<!-- If you want this to go a little bit faster on subsequent runs, you can remove the `render_mode = "human"` part of the `gym.make` call. -->

In [ ]:
problem_2_config = config.copy()                                        # we'll do this a lot when we need to change settings                                        
env = gym.make("CartPole-v1", render_mode = "human")                    # creates the CartPole-v1 environment and renders episodes in it
cartpole_random_policy = random_policy(env)                             # creates a random policy for the environment
trajectory = run_episode(env, cartpole_random_policy, problem_2_config) # your run_episode function will run an episode
env.close()

Now, run 100 episodes of `CartPole-v1` and report the average return of the episodes.

If you find yourself running this function many times, removing the argument `render_mode = "Human"` from the declaration of the environment will speed the process up.

In [ ]:
average_random_return = 0 
env = gym.make("CartPole-v1", render_mode = "human")                                                                                      


# TODO Run 100 episodes of Cartpole using a random policy and then return the average reward you go, you should print it below.
####################################### 



####################################### 


print("The average return of the random policy in CartPole-v1 was {:3.4f}".format(average_random_return))

env.close()

## Problem 3 : 35 points

Great! Now that we have that working, we can move on to the next step: the learning! Like we discussed during the lecture, we're going to start off with *tabular* q-learning.

Thanks to one of the Masters students here at UMass, we have an easy-to-import, gym-ready environment: sepsis treatment management! That's what `sepsis_mdp_master` and `yaml` in our declarations are for.

To be more specific, we're going to use a simulator to learn a policy for treating patients with sepsis, starting by creating a tabular action-value function ($q$) that we can learn with. Recall from the lecture that a tabular $q$ approximator is a table with entries for every pair of state and action $(s,a)$ which contains approximations of

$$
q^{\pi}(s,a) = \mathbb{E}\left[\left.\sum_{t=0}^{\infty}  \gamma^{t} \cdot R(s_{t}, a_{t}) \right| s_{0} = s, a_{0} = a, \pi \right];
$$
the *expected* \[discounted\] return of a trajectory that starts with $s$ and $a$ and continues following the `policy`, $\pi$. Notice that this is equivalent to looking at the average *return to go* starting from a state-action pair. Don't worry too much about the role of $\pi$ in the state-action value function for now, we'll talk about that later.

<!-- (If you're curious, though, we will use what is called the $\epsilon$-greedy policy of the our state-value function approximator `tabular_q`.) -->

## Problem a) 20/35 points
We'll start off by defining the class `tabular_q`. 

In the initialization, you'll need to define a table for the current state-action approximation for each state action pair `state, action`, which will represent the $q$ function. This can get kind of messy, so we have provided you with the number of states in the environment as `self.num_states` and the number of actions as `self.num_actions`.

Also define a pair of functions `tabular_q.get_action` and `tabular_q.probs`, where `tabular_q.get_action` queries `tabular_q.probs` to get the probability of each action under an $\epsilon$-greedy policy, and then selects and outputs that action. There are a few complications with this environment, so I've left in some of the code for you. In particular, do not change the line `return action_index_to_binary(action)` in `get_action` - as far as your code is concerned, the actions in this process are discrete and indexed by the integers $0,1,2,3,4,5,6,7$.

In [ ]:
class tabular_q: # A tabular q approximator
    def __init__(self, environment, configuration):
        self.epsilon = configuration["epsilon"]
        self.stochastic = configuration["stochastic"]

        self.env = environment

        self.num_states = self.env.observation_space.n
        self.num_actions = get_size(self.env.action_space)

        # TODO  Create an array self.state_action_values which stores the current estimate of the state-action value of each state-action pair and
        # a second array self.state_action.samples which stores the number of times each state-action pair has been observed
        ########################################  

        
        ########################################  

    def get_action(self, state, stochastic = True): # function which uses self.probs to sample an action from from a distribution based on the q function or gets the greedy action, depending on the value of stochastic.

        probabilities = self.probs(state)                                                                   

        # TODO Use the "probabilities" (scores if you'd prefer) of each action to choose one
        # If self.stochastic is true then you should randomly select using the probabilities provided (look into np.random.choice)
        # Otherwise just pick the largest value
        ########################################  

        
        ########################################  
        
        # Don't touch this!
        return action_index_to_binary(action)                                                               

    def probs(self, state): # function which gets the epsilon-greedy probabilities associated with the current q-function approximation
        # This function will take a state and produce a probability for each action based off of the "epsilon greedy strategy"
        # This should give the best action a probability of 1 - epsilon and all others should add up to epsilon

        # TODO Calculate the probabilities of each action using the "epsilon greedy" strategy
        ########################################  
        probabilities = np.zeros(self.num_actions)                                                          
        action_values = self.state_action_values[state]                                                     
        best_index = np.argmax(action_values)                                                               
        probabilities[best_index] = 1 - self.epsilon                                                        
        probabilities = probabilities + np.ones(len(probabilities)) * self.epsilon / self.num_actions       
        ########################################  
        return probabilities                                                                                # leave in

### These are functions that you'll use for part 3, just run the block and come back when you need to

In [ ]:
# these are some functions you'll use for part 3.
with open('mdps/configs/gumbel_max_diabetes/homework_config.yml', 'r') as f: # This code creates the sepsis environment.
    env = None
    sepsis_config = yaml.safe_load(f)
    env = gym.make('mdps/GumbelMax-v1', config=sepsis_config)

worst_case = random_policy_sepsis(env)                                      # creates a random policy for comparison                                                                      
tabular = tabular_q(env, config)                                            # initializes the q function you'll train

def test_survival_rate(policy, environment, configuration, suppress_prints = False): # utility function
    sample_size = configuration["survival_rate_sample_size"]                # sample size for evaluation

    deaths = 0                          # counts the number of patients who pass away
    still_in_the_hospital = 0           # counts the number of patients in the hospital at the end of the treatment period
    released = 0                        # counts the number of patients who left the hospital (alive) during the treatment period
    survival = 0                        # counts the number of patients who survive

    test_config = configuration.copy()
    test_config["time_limit"] = configuration["sepsis_time_limit"] # this is very important - otherwise it'll leave the simulated patients in the hospital for years
    test_config["stochastic"] = False   # these two lines should change your policy so that it is not stochastic (as it needs to be for learning) when you're testing it on patients.

    for i in range(sample_size):
        sepsis_trajectory = run_episode(environment, policy, test_config)
        if sepsis_trajectory[len(sepsis_trajectory) - 1][3] == 1: # discounting rewards in processes like this in which only one reward is given at the end does not make a lot of sense.
            released = released + 1
        if sepsis_trajectory[len(sepsis_trajectory) - 1][3] == -1:
            deaths = deaths + 1

    still_in_the_hospital = sample_size - released - deaths
    survival = released + still_in_the_hospital

    if suppress_prints == False:
        print("You treated", sample_size, "patients. In the end,", survival, "patients survived,", still_in_the_hospital, "patients were still in the hospital after 30 days, and", deaths, "patients died.")
        survival_rate = survival / sample_size

        print("The survival (released) rate was", "{:.4f}".format(survival / sample_size), "(", "{:.4f}".format(released / sample_size), ")", ", but of those who survived,", "{:.4f}".format(still_in_the_hospital / survival), "were still in the hospital after a month.")
        if survival_rate < 0.15:
            print("I hope you have malpractice insurance.") # roasted

    return released, still_in_the_hospital, deaths

def plot_stats(stats, line_labels, plot_label, configuration, filename, ylim = False, save = False): # basic plotting function
    if ylim != False:
        plt.ylim(ylim[0], ylim[1])

    for stat in range(len(stats)):
        plt.plot(stats[stat], label = line_labels[stat])

    plt.title(plot_label)
    plt.legend()

    if save == True:
        plt.savefig(plot_label + filename + ".pdf")

    plt.show()

    plt.clf()

def compare_learning_methods(learning_algorithm, environment, configuration, options, slow = False): 
    # function which compares two learning algorithms which are activated by different values of configuration["tabular_q_mode"] - stored in the argument options
    line_labels = ["Released", "Hospital", "Died"] # labels for the graph

    if slow == True: # This runs a much more thorough evaluation process. Use it once your code works with the fast version.
        configuration["survival_rate_sample_size"] = 1000
        training_epochs = 100
        episodes_per_epoch = 100

    else:
        configuration["survival_rate_sample_size"] = 100
        training_epochs = 100
        episodes_per_epoch = 100
    configuration["time_limit"] = configuration["sepsis_time_limit"]

    # set up the policies and the statistics for later plotting
    first_policy = tabular_q(environment, configuration)
    first_stats = np.zeros((training_epochs, len(line_labels)))

    second_policy = tabular_q(environment, configuration)
    second_stats = np.zeros((training_epochs, len(line_labels)))

    # runs the number of training epochs
    for i in range(training_epochs):

        # print information on the current training epoch
        string = "Training epoch " + str(i) + " out of " + str(training_epochs) + " comparing " + options[0] + " to " + options[1]
        print(string, end = '\r')

        string2 = string + " --- " + " Training " + str(options[0])
        print(string2, end = '\r')

        # train
        configuration["tabular_q_mode"] = options[0]
        learning_algorithm(env, first_policy, episodes_per_epoch, configuration)

        # every 25 episodes, print to update with the current performance from test_survival_rate - these can get pretty spammy otherwise
        if i % 25 == 0:
            print(string)
            first_stats[i] = test_survival_rate(first_policy, environment, configuration, suppress_prints = False)
        else:
            first_stats[i] = test_survival_rate(first_policy, environment, configuration, suppress_prints = True)

        # more status updates
        string2 = string + " --- " + " Training " + str(options[1])
        print(string2, end = '\r')

        # train
        configuration["tabular_q_mode"] = options[1]
        learning_algorithm(env, second_policy, episodes_per_epoch, configuration)
        
        # every 25 episodes, print to update with the current performance from test_survival_rate - these can get pretty spammy otherwise
        if i % 25 == 0:
            second_stats[i] = test_survival_rate(second_policy, environment, configuration, suppress_prints = False)
        else:
            second_stats[i] = test_survival_rate(second_policy, environment, configuration, suppress_prints = True)
    
    # plots graphs of the two methods' survival rates side by side
    plot_stats((first_stats / configuration["survival_rate_sample_size"]).T, line_labels, options[0], configuration, "_Sepsis_Training_Graph", [0, 1], save = True)
    plot_stats((second_stats / configuration["survival_rate_sample_size"]).T, line_labels, options[1], configuration, "_Sepsis_Training_Graph", [0, 1], save = True)
    print("Finished! Look at the graphs below to formulate your answer for problem 3b!")

### Problem 3b) 10 Points
Now we can actually get down to business - it's time to build our first learning algorithm. I've already built a competing algorithm, which is run when `learn_tabular_q` is run on a `config` dictionary with `config["tabular_q_mode"] = "Expected_SARSA"`.

Your job is to build the `config["tabular_q_mode"] = "Monte_Carlo"` learning algorithm. Afterward, we'll check out the results and compare the learning methods.

As we spoke about in class, in our case, "Monte Carlo" just denotes the fact that we'll use sample averages to approximate the $q$-values we're after. The naive implementation of sample averaging will be very memory inefficient, so try to use something else. Regardless, the effect of your method should be that if the learning algorithm has seen three instances of a trajectory containing a state-action pair $(s,a)$, with next states $s_{1}$, $s_{2}$, and $s_{3}$ respectively,
$$
(s,a,s_{1},r_{1},G_{1}),\\ (s,a,s_{2},r_{2},G_{2}),\\ (s,a,s_{3},r_{3},G_{3}),
$$
where $G_{i}$ is the return-to-go from the state-action pair $(s,a)$ (including $r_{i}$) in that trajectory, the approximated action-value for that pair should be
$$
\frac{G_{1} + G_{2} + G_{3}}{3}.
$$
Notice that for this algorithm, we don't care about the next state $s'$ or action $a'$ - the Monte Carlo algorithm only considers the true sampled values of the return to go.

In [ ]:
def learn_tabular_q(env, policy, num_episodes, configuration): # function which learns the q approximator policy for num_episodes episodes.
    alpha = configuration["alpha"]
    gamma = configuration["gamma"]

    if configuration["tabular_q_mode"] == "Monte_Carlo": # this is the mode that you'll write
        for episode in range(num_episodes):                                                             
            trajectory = run_episode(env, policy, configuration)                                        

            for t in range(len(trajectory)):                                                            
            # Write code to update your approximation of the q-value for each state-action pair in the trajectory.

                # TODO First, get your state, action, and return_to_go for time step t
                # Action is already done for you since it requires converstion to int, the others don't though
                ########################################  



                ####################################### 

                # TODO Next, you'll need to update your policy, in particular you need to update the following:
                # 1: policy.state_action_values[state][action] -> you should update this using the monte_carlo strategy the values above and 2
                # 2: policy.state_action_samples[state][action] -> This will hold the number of times you've seen this state action pair, this will need to increase here 
                ########################################  



                ########################################  

            print("Episode", episode, "return", trajectory[0][3], end='\r')                             

    elif configuration["tabular_q_mode"] == "Expected_SARSA": # this is the method you'll compare against
        for episode in range(num_episodes):
            trajectory = run_episode(env, policy, configuration)
            
            for t in range(len(trajectory) - 1):
                state = trajectory[t][0]
                action = binary_action_to_int(trajectory[t][1])
                return_to_go = trajectory[t][3]
                num_samples = policy.state_action_samples[state][action]
                reward = trajectory[t][2]
                next_state = trajectory[t + 1][0]

                # You can also use an exponential rolling average for Expected SARSA (and for Monte Carlo). If you change this, also change the averaging method for the last state!

                # policy.state_action_values[state][action] = (1 - alpha) * policy.state_action_values[state][action] + alpha * (reward + gamma * np.dot(policy.probs(next_state), policy.state_action_values[next_state]))
                policy.state_action_values[state][action] = (num_samples * policy.state_action_values[state][action]) / (num_samples + 1) + (reward + gamma * np.dot(policy.probs(next_state), policy.state_action_values[next_state])) / (num_samples + 1)
                
                policy.state_action_samples[state][action] = policy.state_action_samples[state][action] + 1

            final_state = trajectory[-1][0]
            final_action = binary_action_to_int(trajectory[-1][1])
            final_reward = trajectory[-1][2]
            num_samples = policy.state_action_samples[final_state][final_action]

            # If you decide to swap to exponential moving averaging, do so here as well!
            # policy.state_action_values[final_state][final_action] = (1 - alpha) * policy.state_action_values[final_state][final_action] + alpha * final_reward
            policy.state_action_values[final_state][final_action] = (num_samples * policy.state_action_values[final_state][final_action]) / (num_samples + 1) + final_reward / (num_samples + 1)
            
            policy.state_action_samples[final_state][final_action] = policy.state_action_samples[final_state][final_action] + 1


            # The way that SARSA works is different from Monte Carlo: because SARSA relies on the next state, the final state has to be treated separately; there is no next state to consider the expected state-action-value of.

            print("Episode", episode, "return ", "{:.4f}".format(trajectory[0][3]), end = '\r')

Let's begin by testing your code to see how well it does. Use the included `test_survival_rate(policy, environment, configuration)` function. You should see the survival rate get above 0.5 after fewer than 2000 episodes.

In [ ]:
### Verify that your learning algorithm works here - I recommend doing a loop and evaluating the survival rate every 50-200 episodes.





Finally, let's talk about the method of action-value approximation that was implemented above: Expected SARSA.

Like the Monte Carlo algorithm that we implemented, 

Now, let's compare the two methods:

In [ ]:
problem_3_config = config.copy()
problem_3_config["time_limit"] = config["sepsis_time_limit"]
problem_3_config["alpha"] = 0.01
problem_3_config["train_sepsis"] = True 

comparison_options = ["Monte_Carlo", "Expected_SARSA"]
compare_learning_methods(learn_tabular_q, env, problem_3_config, comparison_options, slow = False)

### Problem 3c) : 10 points

### i) 5 points

After you got the above code working properly, you likely noticed something: a lot of the patients that survived were still in the hospital after a month. In our case, this is in large part because of some simplifications used to create the simulator, but it's also a serious possibility in the real world. One potential culprit is the reward function:

In our case, the total return of an episode of treatment was -1 if the patient passed away, 0 if the patient was still in the hospital, and 1 if the patient left the hospital. How do you think these rewards produce this behavior? Answer below.



### ii) 5 points

You may also have noticed some differences between the training algorithms - in some of my experiments, `Expected_SARSA` was faster to learn, but also tended to "thrash" around, sometimes suddenly becoming worse at treatment. That is not ideal, especially for an application as sensitive as medicine. If that happens, it's probably because the update rule for the Expected SARSA algorithm uses *backups*. In other experiments, they appeared to learn at a similar rate.

We discussed the Bellman Equation(s) in class - equations that relate the expected return of the current state to that of the next state:
$
\begin{align}
q^{\pi}(s,a) = R(s,a) &+ \mathbb{E}\left[\left. \sum_{t = 1}^{\infty} \gamma^{t} R(s_{t},a_{t}) \right| s_{0} = s, a_{0} = a, \pi \right]\\% = R(s,a) + \mathbb{E}[G_{s,a}]\\
q^{\pi}(s,a) = R(s,a) &+ \gamma\underset{{\underset{a' \sim \pi(s)}{s' \sim p(s' | s, a);}}}{\mathbb{E}}\left[ q^{\pi}(s', a')\right].
\end{align}
$
For the action-value function $q^{\pi}$, the Bellman equation is given above. The key to the faster convergence (and bumpier performance) of Expected SARSA lies in its update step: each time a state-action pair is observed, the tabular $q$ function approximation $q^{*}$ is updated using the true state, action, reward and next state $s, a, R(s,a), s'$, while averaging over the possible next actions:
$$
q^{*}(s,a) \gets \frac{samples - 1}{samples} q^{*}(s,a) + \frac{1}{samples} \gamma \left(R(s,a) + \underset{a' \sim \pi(s',a)}{\mathbb{E}}\left[q^{*}(s',a)\right]\right).
$$
For final states, a slightly different procedure is followed. In either case, this is called a *backup* step - rather than using the reward-to-go that was actually observed in the trajectory, we approximate the reward to go using other parts of our $q^{*}$ function. If the reward-to-go varies a lot between different episodes, using the true average can be pretty noisy. Backups can help to correct that, and they can give a more "consistent" action-value approximation; since the true action-value function satisfies the Bellman equation, updating our approximation using other values from the same function can bring it closer to satisfying the Bellman equation (and hopefully, to the real action-value function).

Compare the graphs that you got for the two methods and propose an explanation for the differences you see.



### iii) Bonus: 10 points

If you look in to the code for Expected SARSA given above, you'll see that there are actually *two* update rules implemented for the algorithm - one using regular averaging, and the other using what is called an exponential moving average.

Implement exponential moving averaging for **both** of the methods above, and then run compare_learning_methods again, (being careful not to rerun the cell above), and describe any differences you see between them and their true-average counterparts. Do you think that exponential moving averaging is better or worse than regular averaging?

#### Bonus answer here:

## Question 4) 35 Points
For the last question, we're going to look at something much closer both to what we've been doing in the rest of the class and to some of the cutting-edge engineering in Reinforcement Learning: we're going to learn a $q$-network.

Because the code here is a little bit touchy, we've already written most of it for you. We'll point out what you need to add a little bit later. For now, let's talk about what we're going to do. Before you start, see if you can use CUDA processing to speed up your network with the cell below.

In [ ]:
# q_network specific:
cuda_available = torch.cuda.is_available()
device = 'cuda' if cuda_available else 'cpu' # uses CUDA processing for the neural network if available
if cuda_available:
    print("Torch will use device", device, ", which is a CUDA processor for torch operations")
else:
    print("Torch will use device", device, ", which is not a CUDA processor for torch operations")


### Time for Neural Nets!

The learning algorithm below is a simplification of a method known as DQN - Deep Q Networks, which was very popular a few years ago. We need to make several adjustments from tabular q-learning to make this work, take a look at the code and see if you can work them out!

One interesting thing to notice: if you were to visualize the true $q$-values of the current policy throughout training, you would find something curious: they change over time. Because the $q$-function is defined in terms of the *current* policy $\pi$ (denoted $q^{\pi}$), its values change as the policy changes.

However, we've been using $\epsilon$-greedy policies - policies which select the action $a \in \mathcal{A}$ with highest (approximated) $q^{\pi}$ at each state,
$$
\argmax_{a \in \mathcal{A}}(q^{*}(s, a))
$$
with probability roughly $1 - \epsilon$. That means that every time we change the approximated $q^{*}$, the policy we use changes.

That means that unlike the case in Supervised and Unsupervised learning tasks, the "true" values that we're trying to approximate change throughout the training process. Further, it means that a couple of "incorrect" steps, perhaps caused by noisy information from the process or a couple of bad episodes can totally change the policy, and potentially make it much worse.

To counteract these issues, lots of methods that use deep networks for value function approximation convert the individual episodes of a process into batches of *transitions*, of the form
$$
[s_{t}, a_{t}, r_{t}, s_{t + 1}, a_{t + 1}].
$$
If you read that transition from left to right, you'll see that it's SARSA - the namesake of the backup updates that we've been using.

Using a *buffer* of these transitions, methods like DQN train $q$-networks on batches of transitions in much the same way that a supervised learning algorithm would, trying to correctly approximate the $q$-values of the current policy. These methods typically do a *lot* of batches - often one per state transition. That is, if an episode goes on for 100 timesteps, the approximator will be trained on 100 minibatches. Run the code below to set up the objects you'll need for the buffer.

In [ ]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'terminated'))

class ReplayMemory(object): # courtesy of the PyTorch Organization - creates a transition buffer

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    

# used for some q-network stuff
torch_False = torch.zeros(1, dtype=torch.bool, device = device)
torch_False_cpu = torch.zeros(1, dtype=torch.bool, device = 'cpu')



### The algorithm
Our method is, as stated earlier, a simplification of DQN. The training algorithm is contained in the function `train_q_network`. It begins by initializing a neural network using PyTorch, `q_network`, with input dimension equal to the number of state *features*, and output dimension equal to the number of actions. Then, episodes are sampled and the transitions (see above) are added to a transition buffer, called `transitions`. Once there are more transitions in the buffer than the batch size (`config["q_network_batch_size"]`), training starts: for each transition in the sampled episode, the transition is added to the buffer (`transitions.push(...)`), then the network is updated using the updated buffer (`update_q_network(...)`). Before each episode, the value of $\epsilon$ used to define the policy is decayed slightly: 

`q_network_training.epsilon = configuration["q_network_epsilon_decay"] * q_network_training.epsilon`.

Two different methods (`config["batched_td_method]`) are written, both of which are backup methods: Expected SARSA, which we have already discussed, and a method which is called "Q learning" - in which the backup is based on the approximated value of the *best* action, not a sum of action values weighted according to their probability, as is the case in Expected SARSA.

In either case, a step is made towards the backup value, using an error function from `torch` - I recommend either `MSELoss` or `HuberLoss`.

### Problem 4a): 5 points
The neural network `q_approximator` defined below is almost complete, but it's missing something: the function `ep_greedy_probs` isn't finished (assume when the function is run that `batched` is `False` - the `batched` argument is for dealing with some complexities in `torch`)! Making sure to use `pytorch` tensors, output the $\varepsilon$-greedy probabilities for the approximated action values. The print statement at the end of the block will help you figure out if the function is working properly.

In [ ]:
class q_approximator(nn.Module):
    def __init__(self, env, configuration):
        super(q_approximator, self).__init__()
        
        self.configuration = configuration
        self.env = env
        print("Initializing q network for ", self.env.unwrapped.spec.id)
        
        self.state_space = gym.spaces.utils.flatten_space(self.env.observation_space).shape[0]
        print("The size of the state space of ", self.env.unwrapped.spec.id, "is", self.state_space)
        self.action_space = gym.spaces.utils.flatten_space(self.env.action_space).shape[0]
        print("The size of the action space of", self.env.unwrapped.spec.id, "is", self.action_space)

        self.epsilon = configuration["epsilon"]

        self.hidden_layer_size = configuration["q_network_hidden_layer_width"]
        self.nonlinearity = torch.sigmoid #F.sigmoid
        self.layer_1 = nn.Linear(self.state_space, self.hidden_layer_size)
        self.layer_2 = nn.Linear(self.hidden_layer_size, self.hidden_layer_size)
        self.layer_3 = nn.Linear(self.hidden_layer_size, self.hidden_layer_size)
        self.final = nn.Linear(self.hidden_layer_size, self.action_space)
        self.flatten = nn.Flatten()

    def forward(self, state, batched = False):
        x = self.process_state(state, batched)

        x = self.layer_1(x)
        x = self.nonlinearity(x)
        x = self.layer_2(x)
        x = self.nonlinearity(x)
        x = self.final(x)
        return x

    def process_state(self, state, batched):
        if type(state) == int:
            one_hot = torch.zeros(self.state_space)
            one_hot[state] = 1
            x = one_hot

        elif batched == False:
            x = self.flatten(torch.tensor(state.flatten()).float().unsqueeze(0)).to(device) # flattened unsqueezed tensorized state sent to the same device as the Q-network.

        if batched == True:
            x = state
            if self.configuration["train_sepsis"] == True:
                x = x.to(torch.int64)
                x = F.one_hot(x, 720)
                x = x.to(torch.float)

        if type(x) == torch.Tensor and x.get_device() != device:
            x = x.to(device)
        return x

    def softmax_probs(self, x, dim = 1, batched = False):
        return nn.Softmax(dim)(self.forward(x, batched))

    def ep_greedy_probs(self, x, dim = 1, batched = False):
        if batched: # getting epsilon greedy probs is absurdly difficult because torch does not like taking maxima across only certain parts of a tensor and spitting out a tensor giving their arguments, but I think it's necessary
            probs = torch.zeros((len(x), self.action_space))
            best_tensor = self.forward(x, batched)
            best_action = torch.argmax(best_tensor, dim=1, keepdim=True)

            for i in range(len(x)):
                probs[i][best_action[i]] = 1 - self.epsilon # set probs of best action
            
            probs = probs + torch.tensor(self.epsilon/self.action_space) * torch.ones(len(x), self.action_space) # set probs of all other actions
            return probs

        # Start here:
        
        # TODO Given an input x, you must output the probabilities of each action according to epsilon-greedy
        # Hint: You'll need to use forward to get the output which you can treat as the scores of each action, this should be similar to when you did this above
        ##########################################




        ##########################################


        return probs
        
    def probs(self, x, dim = 1, batched = False): # container for the probability functions that enables switching from config
        if self.configuration["q_network_stochasticity"] == "epsilon_greedy":
            probabilities = self.ep_greedy_probs(x, batched = batched)
        elif self.configuration["q_network_stochasticity"] == "softmax":
            probabilities = self.softmax_probs(x, batched = batched)
        else:
            probabilities = self.ep_greedy_probs(x, batched) # defaults to epsilon greedy

        probabilities = probabilities.to(device) # moves probabilities to device. This was necessary on linux but not windows. No idea why.

        return probabilities
    
    def get_action(self, state, stochastic = True): # selects an action from the distribution, exactly the same way that you did for the tabular q function.
        probs = self.probs(state)
        
        if stochastic == True:
            action = torch.multinomial(input = probs, num_samples = 1)

        elif stochastic == False:
            action = torch.argmax(probs)

        return action

Here we are testing your model on a new environment (Lunar Lander), this will require a new module "Box2D" which you'll have to install. If you're having trouble doing pip install you may have to install Swig (https://sourceforge.net/projects/swig/files/swigwin/), I believe in you to figure it out though!

In [ ]:
temp_env = gym.make("LunarLander-v2", render_mode="human") # If you have trouble with Box2D you can change this environment
test_q = q_approximator(temp_env, config).to(device)
state_sample = temp_env.observation_space.sample()
print("The probabilities associated with action values", test_q.forward(state_sample), "were", test_q.ep_greedy_probs(state_sample))
temp_env.close()

You don't need to work on `update_q_network` at all, it's completely finished for you, but I recommend looking through the code and playing around with the `config["batched_td_method"]` argument which is used here - do you think one loss function works better than the other?

In [ ]:
def update_q_network(q_network, configuration,  transition_buffer, internal_optimizer):
    if len(transition_buffer) < configuration["q_network_batch_size"]: # do not update the network if there are not enough sample transitions to run a full batch.
        return

    transitions = transition_buffer.sample(configuration["q_network_batch_size"]) # sample a batch of transitions from the buffer

    batch = Transition(*zip(*transitions)) # unpack the transitions into a training batch. I have no idea how this works, I just copied it from torch.

    # final states must be treated differently from non-final states, so we separate them out
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, # directly from torch.org
                                          batch.next_state)), device=device, dtype=torch.bool)

    # separate the batch out into tensors containing each of the types
    state_batch =   torch.cat(batch.state)
    action_batch =  torch.cat(batch.action)
    reward_batch =  torch.cat(batch.reward)

    # get the current values from the q approximator
    state_action_values = q_network.forward(state_batch, batched = True)

    # initialize tensors for other values we'll need to calculate
    next_state_values =         torch.zeros(configuration["q_network_batch_size"], device=device, dtype = float)
    next_state_action_qs =      torch.zeros((configuration["q_network_batch_size"], q_network.action_space), device=device, dtype = float)
    next_state_action_probs =   torch.zeros((configuration["q_network_batch_size"], q_network.action_space), device=device, dtype = float)

    # the action values we'll use for the correction are only going to differ from the current on the action which changed.
    correct_action_values = state_action_values.clone()

    # optimizer = torch.optim.Adam(q_network.parameters(), lr = config["LR"], amsgrad = True)

    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    with torch.no_grad():
        non_final_state_action_qs = q_network.forward(non_final_next_states, batched = True)

        next_state_action_qs = next_state_action_qs.type(torch.float)
        next_state_action_qs[non_final_mask] = non_final_state_action_qs.type(torch.float)
        non_final_next_state_action_probs = q_network.probs(non_final_next_states, batched = True)

        next_state_action_probs = next_state_action_probs.type(torch.float)
        next_state_action_probs[non_final_mask] = non_final_next_state_action_probs.type(torch.float)


        # Try changing this in the config! If you want to be adventurous, you could try to write another function (for example, you could use SARSA, rather than expected SARSA, or try to do Monte Carlo (back up your code if you do that - it'll require some reworking))

        if configuration["batched_td_method"] == "SARSA":# expected SARSA
            next_state_values_intermediary = torch.sum(next_state_action_probs * next_state_action_qs, dim = -1)

        elif configuration["batched_td_method"] == "argmax":# q learning
            next_state_values_intermediary = next_state_action_qs.max(1)[0] # torch magic dust from torch.org
        else: print("no method selected for batched_td_method")



        next_state_values = next_state_values.type(torch.float)

        next_state_values[non_final_mask] = next_state_values_intermediary[non_final_mask]

    for sample in range(configuration["q_network_batch_size"]):
        action_index = int(action_batch[sample].cpu().detach().numpy())
        correct_action_values[sample][action_index] = configuration["gamma"] * next_state_values[sample] + reward_batch[sample]


    # Try playing with this!
    criterion = nn.MSELoss()
    # criterion = nn.HuberLoss()


    loss = criterion(state_action_values, correct_action_values)
    internal_optimizer.zero_grad()
    loss.backward()


    # Try playing with this!
    torch.nn.utils.clip_grad_value_(q_network.parameters(), 100) # Something like this is recommended to make sure that enormous gradients don't force the algorithm to take bad steps.
    # You could consider removing this clipping altogether, or changing the clipping parameter.


    internal_optimizer.step()

### Problem 4b) 10 Points

`train_q_network` is missing two important steps: first, trajectories aren't generated yet, and second, the values for the transition buffer aren't defined yet. 

1. After `q_network_training.epsilon = ...`, write code to run an episode with `environment` and assign the trajectory to `trajectory`. Once in every `configuration["visualization_frequency]`, instead run an episode using `visual_environment`, if possible.

2. In the following loop, `for timestep in range(...):`, write code to correctly set `state`, `action`, `return_to_go`,  and `current_reward`, as well as `next_state` and `next_action`, when these are available. (If there is not a next state or next action, set these to `None`.)

In [ ]:
def train_q_network(q_network_training, configuration, environment, num_episodes, visual_environment = None, buffer = None, internal_optimizer = None):

    if internal_optimizer == None:
        print("something is messed up, change the call of train_q_network")

    transitions = ReplayMemory(configuration["transition_buffer_size"]) # create transition buffer
    rolling_array = np.zeros(num_episodes)
    return_array = np.zeros(num_episodes)
    if buffer != None:
        transitions = buffer
    rolling = 0 # initialize rolling reward mean

    for episode in range(num_episodes): 
            
        q_network_training.epsilon = configuration["q_network_epsilon_decay"] * q_network_training.epsilon



        # this code will collect the trajectory for the entire episode
        if episode % configuration["visualization_frequency"] == 0:                             
            if visual_environment != None:                                                      
                trajectory = run_episode(visual_environment, q_network_training, configuration) 
            else:                                                                               
                trajectory = run_episode(environment, q_network_training, configuration)        
        else:                                                                                   
            trajectory = run_episode(environment, q_network_training, configuration)            




        for timestep in range(0, len(trajectory)):
            
            # TODO For each timestep, using the recorded trajectories, get the transition
            # the transition is s,a,r,s',a' -- you should get this for every step besides the last
            ##################################################




            ##################################################




            if timestep == len(trajectory) - 1:
                terminated = torch.ones(1, dtype=torch.bool)
            else: terminated = torch.zeros(1, dtype=torch.bool)
        
            if terminated == torch_False_cpu: # I should make this a function because this is enormous and gross.
                state_tens, action_tens, next_state_tens, reward_tens = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0), torch.tensor(action, dtype=torch.float32, device=device).unsqueeze(0), torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0), torch.tensor(current_reward, dtype=torch.float32, device=device).unsqueeze(0)
            else:
                state_tens, action_tens, next_state_tens, reward_tens = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0), torch.tensor(action, dtype=torch.float32, device=device).unsqueeze(0), None, torch.tensor(current_reward, dtype=torch.float32, device=device).unsqueeze(0)

            transitions.push(state_tens, action_tens, next_state_tens, reward_tens, terminated)


            update_q_network(q_network_training, configuration, transitions, internal_optimizer)

        return_array[episode] = trajectory[0][3]

        # for the first 10 episodes, rolling is the true mean.
        if episode < 10: 
            rolling = np.mean(return_array)
        elif episode >= 10:
                rolling = 0.9 * rolling + 0.1 * trajectory[0][3]

        rolling_array[episode] = rolling

        print("The rolling average return at episode", episode, "was", "{:.4f}".format(rolling), ". The return of that episode was", "{:.4f}".format(trajectory[0][3]), ".", end = '\r')


    return q_network_training, transitions, rolling_array, return_array

### Problem 4c) 10 Points:
Now, run the algorithm on the environment `"CartPole-v1"` and analyze the graph. Your algorithm should be able to learn a reasonably good policy. Optimal performance in this problem is a return of 393 - you should be able find at least some episodes which hit that value. If you can't, look over your code for part **b**.

Analyze the graph. What happened? Did the algorithm's performance suddenly jump or degrade? Why do you think that might have happened?

If you were able to use a CUDA device (the second cell in this homework), try using 1000 episodes for training. If you only had access to cpu, try 200.

Try setting `config["visualize_learning"] = True` - this will render every episode that you train with, and it might help you to better understand what behavior tends to lead to changes in the performance of the agent.

In [ ]:
env_name = "CartPole-v1" # controls the name of the environment for network q learning.

if config["visualize_learning"] == True:
    env = gym.make(env_name, render_mode = "human")
    visual_env = env

else:
    env = gym.make(env_name)
    visual_env = gym.make(env_name, render_mode = "human") 

q_network = q_approximator(env, config).to(device)

optimizer = torch.optim.Adam(q_network.parameters(), lr = config["LR"], amsgrad = True)

x, y, rolling_array, return_array = train_q_network(q_network, config, env, 200, visual_env, internal_optimizer = optimizer)
plot_stats([rolling_array, return_array], ["rolling average", "return"], "Q Network Training Graph for " + env.unwrapped.spec.id, config, "q_network_training_graph" + env.unwrapped.spec.id, save = True)

env.close()

### Problem 4d) 10 points:
Now, we're going to go back to training $q$ for the sepsis problem! How does the $q$-network perform in comparison to your earlier experiments?

This will take a while to train, if its taking too long you can change the model (neural net) above, especially using `config["q_network_batch_size"]` and `config["q_network_hidden_layer_width"]`, but you must run this to show your training works

In [ ]:
# Try the q-network on the sepsis problem from earlier
with open('mdps/configs/gumbel_max_diabetes/homework_config.yml', 'r') as f: # This code creates the sepsis environment.
    env = None
    sepsis_config = yaml.safe_load(f)
    env = gym.make('mdps/GumbelMax-v1', config=sepsis_config)

config_for_sepsis = config.copy()
config_for_sepsis["time_limit"] = config["sepsis_time_limit"]
config_for_sepsis["train_sepsis"] = True

q_network_sepsis = q_approximator(env, config_for_sepsis).to(device)

optimizer = torch.optim.Adam(q_network_sepsis.parameters(), lr = config["LR"], amsgrad = True)

num_rounds = 20
epochs_per_round = 100

stats = np.zeros((num_rounds + 1, 3))
buffer = []
sepsis_rolling_array = np.zeros((num_rounds + 1, epochs_per_round))
sepsis_return_array = np.zeros((num_rounds + 1, epochs_per_round))

q_network_sepsis, buffer, sepsis_rolling_array[0], sepsis_return_array[0] = train_q_network(q_network_sepsis, config_for_sepsis, env, epochs_per_round, internal_optimizer = optimizer)
stats[0] = test_survival_rate(q_network_sepsis, env, config_for_sepsis)

for i in range(1, num_rounds):
    q_network_sepsis, buffer, sepsis_rolling_array[i], sepsis_return_array[i] = train_q_network(q_network_sepsis, config_for_sepsis, env, epochs_per_round, buffer = buffer, internal_optimizer = optimizer)
    stats[i] = test_survival_rate(q_network_sepsis, env, config_for_sepsis)

sepsis_rolling_array = sepsis_rolling_array.flatten()
sepsis_return_array = sepsis_return_array.flatten()

plot_stats([sepsis_rolling_array, sepsis_return_array], ["rolling average", "return"], "Q Network Training Graph for " + "sepsis", config_for_sepsis, "q_network_training_graph" + "sepsis", save = True)


line_labels = ["Released", "Hospital", "Died"] # labels for the graph
plot_stats((stats / config_for_sepsis["survival_rate_sample_size"]).T, line_labels, "q-network Sepsis", config_for_sepsis, "_Sepsis_Training_Graph", [0, 1], save = True)

env.close()

## Problem 5) Bonus: 15 points

Pick any environment from [gymnasium](https://gymnasium.farama.org/environments/classic_control/) and fiddle with the parameters of the learning algorithm (and model itself) until you have a good training graph and good performance at the end of training. Explain your process and show graphs of performance below. Consider changing the parameters in `config["q_network_batch_size"]`, `config["epsilon"]` and `config["q_network_hidden_layer_width"]` as well.
